In [ ]:
import cv2
import numpy as np 
kernel = np.ones((3,3),np.uint8)
vc = cv2.VideoCapture(0)

while vc.isOpened():
    rval, frame = vc.read()    # read video frames again at each loop, as long as the stream is open
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    thr_value, img_thresh = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY_INV)
    img_close = cv2.morphologyEx(img_thresh, cv2.MORPH_CLOSE, kernel, iterations = 2)
    img_canny = cv2.Canny(img_close, 100, 200)    # standard canny edge detector
    contours, hierarchy = cv2.findContours(img_canny,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #contours is not an image, is a chain of pixel locations
    cv2.drawContours(frame, contours,-1,(0,255,0),2) # paint contours on top of original image
    cv2.imshow("stream", frame)# display each frame as an image, "stream" is the name of the window
    key = cv2.waitKey(1)       # allows user intervention without stopping the stream (pause in ms)
    if key == 27:              # exit on ESC
        break
        
cv2.destroyWindow("stream")    # close image window upon exit
vc.release()     

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
img = cv2.imread('./images/unocard.jpg')

kernel_size = 3
kernel=np.ones((kernel_size,kernel_size),np.uint8)


thr_value, img_thresh = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY_INV)
img_close = cv2.morphologyEx(img_thresh, cv2.MORPH_CLOSE, kernel, iterations = 1)
img_canny = cv2.Canny(img_close, 100, 230)    # standard canny edge detector
contours, hierarchy = cv2.findContours(img_canny,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #contours is not an image, is a chain of pixel locations

cv2.drawContours(img, contours,-1,(0,255,0),3) # paint contours on top of original image
imS = cv2.resize(img,(600, 600))
cv2.imshow('contours', imS)
cv2.waitKey(0)
cv2.destroyAllWindows() 
# smooth = cv2.blur(img, (3,3))
# thresh =  cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,15,2)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
img = cv2.imread('./images/unocard.jpg',0)

kernel_size = 3
kernel=np.ones((kernel_size,kernel_size),np.uint8)
thr_value, img_thresh = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)

dilation = cv2.dilate(img_thresh,kernel,iterations=1)
erosion = cv2.erode(img_thresh,kernel,iterations=1)
opening  = cv2.morphologyEx(img_thresh,cv2.MORPH_OPEN,kernel)
gradient = cv2.morphologyEx(img_thresh,cv2.MORPH_GRADIENT,kernel)
closing = cv2.morphologyEx(img_thresh,cv2.MORPH_CLOSE,kernel)


titles = ['Binary','dilate','erode','close','open','gradient']
images = [img_thresh,dilation,erosion,closing,opening,gradient]

for i in range(len(images)):
    plt.subplot(2, 3, i + 1), plt.imshow(images[i], 'gray')
    plt.title(titles[i])
    plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
import cv2
import numpy as np

# Load the image of the UNO card and the template to match
img = cv2.imread('uno_card.jpg', cv2.IMREAD_GRAYSCALE)
template = cv2.imread('uno_template.jpg', cv2.IMREAD_GRAYSCALE)

# Get the width and height of the template
w, h = template.shape[::-1]

# Perform template matching using cv2.TM_CCOEFF_NORMED method
res = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)

# Set a threshold for the similarity score to consider a match
threshold = 0.8

# Find the location of the match(es) above the threshold
loc = np.where(res >= threshold)

# Draw a rectangle around the matching region(s) on the original image
for pt in zip(*loc[::-1]):
    cv2.rectangle(img, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)

# Display the result
cv2.imshow('result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Contouring for the card images using custom thresholding

In [24]:
import cv2
import numpy as np
img_colour = cv2.imread('./unocards/y1.jpg')   # open the saved image in colour

img = cv2.cvtColor(img_colour, cv2.COLOR_BGR2GRAY)   # convert to B/W
img_HSV = cv2.cvtColor(img_colour, cv2.COLOR_BGR2HSV) # Convert to HSV
img_sm = cv2.blur(img, (7, 7))         # smoothing
thr_value, img_th = cv2.threshold(img_sm, 160, 255, cv2.THRESH_BINARY_INV)   # binarisation
# img_th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 35, 2)
kernel = np.ones((3, 3), np.uint8)
img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)      # morphology correction
img_canny = cv2.Canny(img_close, 100, 200)                          # edge detection
contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)   # extract contours on binarised image, not on canny
cv2.drawContours(img_colour, contours, -1, (0, 255, 0), 2)         # paint contours on top of original coloured mage
cv2.imshow('color with contours',img_colour)
cv2.imshow('thresholded', img_th)
cv2.imshow('canny', img_canny)
key = cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
# imgc = cv2.imread('./unocards/r6.jpg')    # open the saved image in colour 
for i, c in enumerate(contours):         # loop through all the found contours
    print(i, ':', hierarchy[0, i])          # display contour hierarchy
    print('length: ', len(c))               # display numbr of points in contour c
    perimeter = cv2.arcLength(c, True)     # perimeter of contour c (curved length)
    print('perimeter: ', perimeter)               
    epsilon = 5 # 0.01*perimeter    # parameter of polygon approximation: smaller values provide more vertices
    vertex_approx = len(cv2.approxPolyDP(c, epsilon, True))     # approximate with polygon
    print('approx corners: ', vertex_approx, '\n')                    # number of vertices
    cv2.drawContours(img_colour, [c], 0, (0, 255, 0), 3)   # paint contour c
    cv2.putText(img_colour, str(i), (c[0, 0, 0]+20, c[0, 0, 1]+30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255))    # identify contour c
    [x,y,w,h] = cv2.boundingRect(c)
    cv2.rectangle(img_colour, (x,y), (x+w,y+h), (255, 0, 0), 2)
# cv2.namedWindow('picture', cv2.WINDOW_NORMAL)
cv2.imshow('picture',img_colour)
key = cv2.waitKey(0)
cv2.destroyAllWindows()

0 : [-1 -1  1 -1]
length:  2236
perimeter:  2236.0
approx corners:  4 

1 : [-1 -1  2  0]
length:  1276
perimeter:  1305.82337474823
approx corners:  8 

2 : [ 4 -1  3  1]
length:  187
perimeter:  199.8406196832657
approx corners:  7 

3 : [-1 -1 -1  2]
length:  132
perimeter:  141.94112491607666
approx corners:  5 

4 : [6 2 5 1]
length:  391
perimeter:  415.4385987520218
approx corners:  11 

5 : [-1 -1 -1  4]
length:  314
perimeter:  328.911687374115
approx corners:  8 

6 : [-1  4  7  1]
length:  189
perimeter:  201.01219260692596
approx corners:  8 

7 : [-1 -1 -1  6]
length:  134
perimeter:  143.94112491607666
approx corners:  6 



In [25]:
contour1 = contours[1]

# Find the bounding rectangle of contour 1
x, y, w, h = cv2.boundingRect(contour1)

# Crop the image to the bounding rectangle
crop_img = img_colour[y:y+h, x:x+w]

# Display the cropped image
cv2.imshow('Contour 1', crop_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
# Convert the image to HSV
hsv = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)

# Define the range of colors for the Uno card
lower_red = np.array([0,50,50])
upper_red = np.array([10,255,255])

# Create a mask for the red color
mask = cv2.inRange(hsv, lower_red, upper_red)

# Bitwise-AND mask and original image
res = cv2.bitwise_and(crop_img,crop_img, mask= mask)

# Check if the card is red
if np.any(res):
    print("The Uno card is red")
else:
    print("The Uno card is not red")

The Uno card is red


Getting Contours using colour thresholding

In [ ]:
import cv2
import numpy as np
img_colour = cv2.imread('./unocards/r6.jpg')   # open the saved image in colour
####
# Add something to detect color
####
HSVmin = np.array([0, 150, 170])
HSVmax = np.array([10, 255, 255])
img_HSV = cv2.cvtColor(img_colour, cv2.COLOR_BGR2HSV)  # Convert to HSV
img_th = cv2.inRange(img_HSV, HSVmin, HSVmax) 
# img = cv2.cvtColor(img_colour, cv2.COLOR_BGR2GRAY)   # convert to B/W

img_sm = cv2.blur(img_HSV, (5, 5))         # smoothing
# thr_value, img_th = cv2.threshold(img_sm, 160, 255, cv2.THRESH_BINARY_INV)   # binarisation
# img_th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 35, 2)
kernel = np.ones((3, 3), np.uint8)
img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)      # morphology correction
img_canny = cv2.Canny(img_close, 100, 200)                          # edge detection
contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)   # extract contours on binarised image, not on canny
cv2.drawContours(img_colour, contours, -1, (0, 255, 0), 2)         # paint contours on top of original coloured mage
cv2.imshow('color with contours',img_colour)
cv2.imshow('thresholded', img_th)
cv2.imshow('canny', img_canny)
key = cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
print(len(contours))

In [ ]:
print(contours)

In [9]:
imgc = cv2.imread('./unocards/r6.jpg')    # open the saved image in colour 
for i, c in enumerate(contours):         # loop through all the found contours
    print(i, ':', hierarchy[0, i])          # display contour hierarchy
    print('length: ', len(c))               # display numbr of points in contour c
    perimeter = cv2.arcLength(c, True)     # perimeter of contour c (curved length)
    print('perimeter: ', perimeter)               
    epsilon = 5 # 0.01*perimeter    # parameter of polygon approximation: smaller values provide more vertices
    vertex_approx = len(cv2.approxPolyDP(c, epsilon, True))     # approximate with polygon
    print('approx corners: ', vertex_approx, '\n')                    # number of vertices
    cv2.drawContours(imgc, [c], 0, (0, 255, 0), 3)   # paint contour c
    cv2.putText(imgc, str(i), (c[0, 0, 0]+20, c[0, 0, 1]+30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255))    # identify contour c
    [x,y,w,h] = cv2.boundingRect(c)
    cv2.rectangle(imgc, (x,y), (x+w,y+h), (255, 0, 0), 2)
# cv2.namedWindow('picture', cv2.WINDOW_NORMAL)
cv2.imshow('picture',imgc)
key = cv2.waitKey(0)
cv2.destroyAllWindows()

0 : [ 4 -1  1 -1]
length:  706
perimeter:  783.8721451759338
approx corners:  12 

1 : [-1 -1  2  0]
length:  203
perimeter:  231.5807341337204
approx corners:  7 

2 : [ 3 -1 -1  1]
length:  3
perimeter:  3.414213538169861
approx corners:  1 

3 : [-1  2 -1  1]
length:  23
perimeter:  25.899494767189026
approx corners:  2 

4 : [ 8  0  5 -1]
length:  726
perimeter:  857.7199051380157
approx corners:  16 

5 : [-1 -1  6  4]
length:  476
perimeter:  542.2741661071777
approx corners:  15 

6 : [ 7 -1 -1  5]
length:  36
perimeter:  40.97056245803833
approx corners:  3 

7 : [-1  6 -1  5]
length:  69
perimeter:  77.69848430156708
approx corners:  4 

8 : [-1  4  9 -1]
length:  703
perimeter:  779.6295045614243
approx corners:  10 

9 : [-1 -1 10  8]
length:  211
perimeter:  238.75230705738068
approx corners:  11 

10 : [11 -1 -1  9]
length:  1
perimeter:  0.0
approx corners:  1 

11 : [-1 10 -1  9]
length:  21
perimeter:  23.899494767189026
approx corners:  2 

